In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime

In [2]:
# Read Global Temperatures by Country file and store into Pandas data frame
land_temp_by_country_csv = "../../ProjectOne_datasets/GlobalLandTemperaturesByCountry.csv"
land_temp_by_country_df = pd.read_csv(land_temp_by_country_csv)
land_temp_by_country_df.tail()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe
577461,2013-09-01,NaN,NaN,Zimbabwe


In [3]:
# Read Population Growth by Country file and store into Pandas data frame
population_growth_country_csv = "../../ProjectOne_datasets/population-by-country.csv"
population_growth_country_df = pd.read_csv(population_growth_country_csv)
population_growth_country_df.tail()

#Rename Column for World Population data
population_growth_country_df = population_growth_country_df.rename(columns={"Population by Country (Clio Infra (2016))":"Population",
                                                                           "Entity":"Country"})
population_growth_country_df.head()

,Country,Code,Year,Population
0,Afghanistan,AFG,1500,2000000.0
1,Afghanistan,AFG,1600,2500000.0
2,Afghanistan,AFG,1700,2500000.0
3,Afghanistan,AFG,1800,3280000.0
4,Afghanistan,AFG,1820,3280000.0


In [4]:
#Load CO2 Emissions Growth by Country file and store into Pandas data frame
co2_country_csv = "../../ProjectOne_datasets/annual-co-emissions-by-region.csv"
co2_country_df = pd.read_csv(co2_country_csv)
co2_country_df.tail()

#Rename Column 
co2_country_df = co2_country_df.rename(columns={"Entity":"Country"})
co2_country_df.head()

,Country,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1750,0.0
1,Afghanistan,AFG,1751,0.0
2,Afghanistan,AFG,1752,0.0
3,Afghanistan,AFG,1753,0.0
4,Afghanistan,AFG,1754,0.0


In [5]:
# Cleanup Land Temperature Datasets
########################################

#Drop AverageTemperatureUncertainty
land_temp_by_country_df = land_temp_by_country_df.drop(["AverageTemperatureUncertainty"], axis=1)

#Remove rows with null temperatures
land_temp_by_country_df = land_temp_by_country_df[land_temp_by_country_df["AverageTemperature"].notna()]

print(f"Country Data Size: {len(land_temp_by_country_df)}")

Country Data Size: 544811


In [6]:
#Retrieve rows for the last century
cent_land_temp_by_country_df = land_temp_by_country_df.loc[land_temp_by_country_df["dt"] >= "1900-01-01"]
clean_co2_df=co2_country_df.loc[co2_country_df['Year'] >= 1900]

In [7]:
# Function to get Surface Temperatures for given country
global countries_temperatures_df 
columns = ["Decade", "Land Temperature", "Country"]
countries_temperatures_df = pd.DataFrame(columns = columns)

def get_surface_temps_by_country(country):
    global countries_temperatures_df 
    temp_df = cent_land_temp_by_country_df.loc[land_temp_by_country_df["Country"]==country]
    temp_df = temp_df.groupby((pd.DatetimeIndex(temp_df["dt"]).year//10)*10).mean()
    temp_df["Country"] = country

    #Reset index, and rename columns appropriately
    temp_df.reset_index(inplace=True)
    temp_df = temp_df.rename(columns = {'dt':'Decade'})
    temp_df = temp_df.rename(columns = {'AverageTemperature':'Land Temperature'})
    temp_df = temp_df.loc[temp_df["Decade"] != 2010]
    temp_df
    countries_temperatures_df = pd.concat([countries_temperatures_df, temp_df], axis=0)

In [8]:
# Function to get Population for given country
global countries_population_df 
columns = ["Decade", "Population", "Country"]
countries_population_df = pd.DataFrame(columns = columns)

def get_population_by_country(country):
    global countries_population_df 
    
    temp_df = population_growth_country_df.loc[population_growth_country_df["Country"] == country]
    temp_df = temp_df.loc[temp_df["Year"] >= 1900]
    temp_df = temp_df.rename(columns = {'Year':'Decade'})
    temp_df = temp_df.drop(["Code"], axis=1)
    countries_population_df = pd.concat([countries_population_df, temp_df], axis=0)

In [9]:
# Function to get CO2 Emissions for given country
global countries_CO2_df 
columns = ["Decade", "Annual CO2 emissions", "Country"]
countries_CO2_df = pd.DataFrame(columns = columns)

def get_CO2_by_country(country):
    global countries_CO2_df 
    
    temp_df = co2_country_df.loc[co2_country_df["Country"] == country]
    temp_df = temp_df.loc[temp_df["Year"] >= 1900]
    temp_df = temp_df.groupby((temp_df["Year"]//10)*10).mean()
    temp_df["Country"] = country
    
    temp_df = temp_df.rename(columns = {'Year':'Decade'})
    temp_df = temp_df.loc[temp_df["Decade"] != 2010]
    countries_CO2_df = pd.concat([countries_CO2_df, temp_df], axis=0)

In [10]:
#list of countries to operate on
list_of_countries = ["United States", "China", "Brazil", "Denmark", "India","United Kingdom"]

for country in list_of_countries:
    get_surface_temps_by_country(country)
    get_population_by_country(country)
    get_CO2_by_country(country)

#clean up CO2 dataframe
countries_CO2_df.reset_index(inplace=True)
countries_CO2_df = countries_CO2_df.drop(["Decade"], axis=1)
countries_CO2_df = countries_CO2_df.rename(columns = {'index':'Decade'})


In [11]:
#Final cleansed and merged datasets
final_population_land_temps_df = pd.merge(countries_temperatures_df, countries_population_df, on=["Decade", "Country"])
final_df = pd.merge(final_population_land_temps_df, countries_CO2_df, on=["Decade", "Country"])
final_df = final_df.rename(columns = {"Annual CO2 emissions":"Annual CO2 Emissions"})
print(f"Merged Data Set lenght: {len(final_population_land_temps_df)}") 

Merged Data Set lenght: 66


In [12]:
#Reorganize the column names for ease of redability
column_names = ["Decade", "Country", "Land Temperature", "Population", "Annual CO2 Emissions"]
final_df = final_df.reindex(columns=column_names)
final_df

,Decade,Country,Land Temperature,Population,Annual CO2 Emissions
0,1900,United States,8.448225,76391000.0,9.390961e+08
1,1910,United States,8.460250,92767000.0,1.447061e+09
2,1920,United States,8.755250,106881000.0,1.754044e+09
3,1930,United States,9.086992,123668000.0,1.543642e+09
4,1940,United States,8.924092,132637000.0,2.264069e+09
...,...,...,...,...,...
61,1960,United Kingdom,8.496675,52372000.0,6.043932e+08
62,1970,United Kingdom,8.718533,55632000.0,6.293116e+08
63,1980,United Kingdom,8.721058,56314000.0,5.614150e+08
64,1990,United Kingdom,9.208950,57493307.0,5.802683e+08


In [22]:
# Read Lat Long by Country file and store into Pandas data frame
country_geocode_path = "../../ProjectOne_datasets/countries_geocode.csv"
country_geocode_df = pd.read_csv(country_geocode_path)
country_geocode_df = country_geocode_df.rename(columns = {"name":"Country"})
country_geocode_df = country_geocode_df.drop(["country"], axis=1)
country_geocode_df.tail()

exportable_df = pd.merge(final_df, country_geocode_df, on=["Country"], how="inner")
exportable_df.tail()

,Decade,Country,Land Temperature,Population,Annual CO2 Emissions,latitude,longitude
61,1960,United Kingdom,8.496675,52372000.0,604393213.5,55.378051,-3.435973
62,1970,United Kingdom,8.718533,55632000.0,629311649.7,55.378051,-3.435973
63,1980,United Kingdom,8.721058,56314000.0,561415016.8,55.378051,-3.435973
64,1990,United Kingdom,9.208950,57493307.0,580268341.1,55.378051,-3.435973
65,2000,United Kingdom,9.587658,59522468.0,558479646.5,55.378051,-3.435973


In [21]:
# Push the remade DataFrame to a new CSV file
exportable_df.to_csv("../Output/Population_SurfaceTemps_CO2_By_Country.csv",
                  encoding="utf-8", index=False, header=True)